In [ ]:
#|default_exp write_nb

# Write Notebooks
> A module to create new notebooks quickly

In [ ]:
#|export
from string import Template
import json
from fastcore.script import call_parse
from fastcore.foundation import L, AttrDict


_default_exp = Template('#|default_exp $export')
_title = Template('# $title\n')
_description = Template("> $description")
_export = "#|hide\n#|eval:false\nfrom nbprocess.doclinks import nbprocess_export\nnbprocess_export()"

def _mk_nb(title,desc,exp=None):
    def _mk_cell(kind,src): return dict(cell_type=kind,metadata={},outputs=[],source=src)
    nb=AttrDict(cells=L(),metadata={},nbformat=4,nbformat_minor=5)
    if exp is not None: nb.cells.append(_mk_cell("code",[exp]))
    nb.cells.append(_mk_cell("markdown",[title,desc]))
    nb.cells.append(_mk_cell("code",[]))
    if exp is not None:
        nb.cells.append(_mk_cell("markdown",["## Export -"]))
        nb.cells.append(_mk_cell("code", _export))
    nb.cells = list(nb.cells)
    return dict(nb)

In [ ]:
#|export
@call_parse
def new_nb(
    name:str, # The name of the newly created notebook
    mod:str=None, # The name of the exported module from it
    title:str=None, # The title header in the notebook
    description:str=None, # The description that will go under the title header
    nprefix:int=0, # If > 0, will be the first number in the filename
    nsuffix:int=0, # If > 0, will be the second number in the filename
):
    "Creates a new base nbprocess notebook {nprefix}{nsuffix}_{name}.ipynb"
    cfg = get_config()
    nbs = nbglob(cfg.nbs_path, file_glob="*.ipynb", file_re="^[0-9]", skip_folder_re="^[_.]")
    nbs = nbs.map(lambda x: x.split("./")[-1].replace(".ipynb",""))
    nbs.sort()
    title = _title.substitute(title=title or "No Title")
    descrip = _description.substitute(description=description or "Fill me in!")
    if mod is not None: mod = _default_exp.substitute(export=mod)
    nums = nbs.map(
        lambda x: list(re.search("^[0-9][0-9]?", x).group(0))
    ).map(lambda x: list(map(int, x)))
    if nprefix==0 and len(nums)>0: nprefix = max(o[0] for o in nums)
    if nsuffix==0 and len(nums)>0: nsuffix = max(o[1] for o in nums)+1
    nbpath = Path(f'{cfg.nbs_path}/{nprefix}{nsuffix}_{name}.ipynb')
    with nbpath.open("w") as f: json.dump(_mk_nb(title,descrip,mod), f, indent=1)